<a href="https://colab.research.google.com/github/micaelaustad/TDT4310_Hint/blob/master/GPT2_the_small_version_%F0%9F%98%87_%F0%9F%A5%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

There are some packages you need, in theory you could actually just use pytorch and transformers (or tensorflow because transformers has two backends 😼 ) 

In [1]:
!pip install nltk
!pip install sklearn
!pip install transformers
!pip3 install torch torchvision


     |████████████████████████████████| 2.1MB 19.1MB/s 
     |████████████████████████████████| 3.3MB 51.5MB/s 
     |████████████████████████████████| 901kB 48.2MB/s 


Drop the following to code blocks if you do not intend to use a GPU to train :D 
Google Colab does not always give a GPU 😢


In [2]:
!nvidia-smi

Mon May  3 08:55:04 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
 import torch
 torch.cuda.set_device(0)
 torch.cuda.current_device()

0

In [4]:
from transformers import Trainer, TrainingArguments
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer,pipeline,AutoModelForCausalLM
import re
import json
from sklearn.model_selection import train_test_split
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

One of the students had used LOTR for another task, so i thought it would be fun to use that as the base for training the LM. 😇

You need to split the data as usually, i'm testing it without shuffling hoping that it will give better flow between sentences.

In [5]:
#You need to add the file (or any other file :S ) to google colab to be able to use this. 
with open('fellowship.txt') as f:
    data = nltk.sent_tokenize(f.read())


def build_text_files(data, dest_path):
    f = open(dest_path, 'w')
    f.writelines(data)


train, test = train_test_split(data, test_size=0.15,shuffle=False)

build_text_files(train, 'train_dataset.txt')
build_text_files(test, 'test_dataset.txt')

print("Train dataset length: "+str(len(train)))
print("Test dataset length: " + str(len(test)))

Train dataset length: 10789
Test dataset length: 1905


Transfear learning models are pretrained, so they also have their own specialized tokenizers. GPT2 is a causal ML so it is not as sensitive to alot of the issues that others are, but in general it is both about what kind of format the data should be on , as well as seperating tokens. (f.ex attention masks 😉) 

In [6]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'


def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=train_path,
        block_size=128)

    test_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=test_path,
        block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator


train_dataset, test_dataset, data_collator = load_dataset(
    train_path, test_path, tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


The only parameters that should be completly unknown is steps, and the warmup, the others are shared with every other DL algorithm. 

A note on batch size tho is that these algorihms are huge, so batch size has alot of impact. 

Also the least intuitive value is warm-up steps, it is just the steps used to adjust for something called early-overfitting. It helps to attune the attention and the optimizer to the problem.

In [7]:
model = AutoModelForCausalLM.from_pretrained("gpt2")
training_args = TrainingArguments(
    output_dir="./gpt2-LOTR",  # The output directory
    overwrite_output_dir=True,  # overwrite the content of the output directory
    num_train_epochs=5,  # number of training epochs
    per_device_train_batch_size=8,  # batch size for training - lower is generally better, but 1 is not adviced, go for something below 10, powers of 2 is the norm. 
    per_device_eval_batch_size=8,  # batch size for evaluation
    eval_steps=400,  # Number of update steps between two evaluations.
    save_steps=1000000,  # after # steps model is saved
    warmup_steps=800  # number of warmup steps for learning rate scheduler, it helps tuning the attenion mechanicms.
)
print("build trainer with on device:", training_args.device, "with n gpus:", training_args.n_gpu)
print(training_args.device)
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)
trainer.train()
trainer.save_model()


build trainer with on device: cuda:0 with n gpus: 1
cuda:0


Step,Training Loss
500,3.655400
1000,3.207600


You can play around with the finished product here, it makes pretty decent text, far from good enough to be a book, but certainly coherent and with some interesting twists and turns. 

In [8]:
from transformers import pipeline


def generate_LOTR(text):
    generator = pipeline('text-generation', model='./gpt2-LOTR',
                         tokenizer='gpt2', config={'max_length': 1000})

    result = map(lambda res: res['generated_text'],
                 generator(text))
    print([print(res) for res in result])


sentences = [
    "Aragorn raised his head slowly",
    "Gandalf fell down to the dark",
    "Legolas notched an arrow ",
    "The darkness covered the landscape",
    "Give me examples please, i am fast i promse ^^"
]
for sent in sentences:
    generate_LOTR(sent)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Aragorn raised his head slowly.In the silence they heard faint whispers as a stream ran down to the low hill-side: many voices that seemed to be calling out in song.Before long they heard the wind above.`We are going
[None]


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Gandalf fell down to the dark pool behind them.The path was flat to the bottom of a steep, well-groomed slope.Frodo and Legolas led him to it.Legolas was still breathing, but it seemed
[None]


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Legolas notched an arrow ering near the tree; for a cold wind whirled round and hung like a shadow over the branches of a fallen tree.They were suddenly aware that one after another was gathering nearer and nearer.Some stepped aside,
[None]


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The darkness covered the landscape, as if the land grew grey and fainter.The night was long, while the birds played and laughed.The light grew in the mistle of the great mounds ahead in the west; and as they went
[None]


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Give me examples please, i am fast i promse ^^

'I am a servant of the Master!'beeped Frodo.`There are at least fifteen of you still left here, and you can have as many servants as
[None]
